In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import re
import os
import seaborn as sns
import pandas as pd
import scipy.signal
from fastai.vision import *

from model import *

In [ ]:
EXP_REF = "t5_reference"
REF_FILE = f"res/{EXP_REF}.txt"
REF_MATRIX_FOLDER = f"matrixes/{EXP_REF}"

## Uncomment the experiment to analyze
#EXP = "t5_init"
#EXP = "t5_feats"
#EXP = "t5_metrics"
FILE_TO_LOAD = f"res/{EXP}.txt"
MATRIX_FOLDER = f"matrixes/{EXP}"

EXP_TAG = EXP[EXP.find('_')+1:]
print(f'Experiment type: {EXP_TAG}')

pthr = 0.05

In [ ]:
def pretty_config(c):
    c = c.rstrip("\n")
    c = re.sub(r'<function (\w+) at \w+>', r"'\1'", c)
    c = re.sub(r'"data": "[^"]+", ', r"", c)
    return c

In [ ]:
full_configs = []
configs = []

In [ ]:
file = open(REF_FILE, 'r')
lines = file.readlines()

for line in lines:
    c = json.loads(pretty_config(line))
    if c['config']['type'] == 'reference': 
        full_configs.append(c)
        configs.append(c['config'])

In [ ]:
file = open(FILE_TO_LOAD, 'r')
lines = file.readlines()

for line in lines:
    c = json.loads(pretty_config(line))
    full_configs.append(c)
    configs.append(c['config'])

In [ ]:
results = np.empty((len(full_configs), 
                    len(full_configs[0]['results']), 
                    len(full_configs[0]['results'][0]['accuracy'])))
for i, c in enumerate(full_configs):
    for j, r in enumerate(c['results']):
        results[i, j, :] = np.array(r['accuracy'])

In [ ]:
ref = None
pvalues = []

for i in range(len(configs)):
    c = configs[i]
    r = results[i, :, -1]
    if ref is None and c['type'] == 'reference':
        ref = r 
    if ref is not None:
        pvalue = scipy.stats.mannwhitneyu(r, ref, alternative='two-sided').pvalue
    else:
        pvalue = 1.0
    pvalues.append(pvalue)
    print('-----------')
    print('Config:', c)
    print(f'Mean: {np.mean(r)*100:.2f}  |  Median: {np.median(r)*100:.2f}')
    print(f'P-value: [{pvalue < pthr}] {pvalue}')
    totalTime = 0
    for j, r in enumerate(full_configs[i]['results']):
        totalTime += r['time']
    print(f'Time: {int(totalTime / 60)} mins ({round(totalTime / 3600, 1)} hours)')

In [ ]:
def convertMetric(order):
    order = order.replace('\'', '')
    order = order.replace('_image_batch', '')
    if order == 'total_variation':
        return r'$\text{TV}$'
    elif order == 'activation_sum':
        return r'$\text{Sum}$'
    elif order == 'entropy':
        return r'$H$'
    elif order == 'entropy_shannon_1diff':
        return r'$H_{sv}$'
    elif order == 'max_activation':
        return r'$\text{Max}$'
    elif order == 'median_activation':
        return r'$\text{Median}$'
    return order
    

In [ ]:
resultsForTable = {}

for i in range(len(configs)):
    c = configs[i]
    r = results[i, :, -1]
    if ref is None and c['type'] == 'reference':
        ref = r 
        pvalue = 1.0
    else:
        pvalue = scipy.stats.mannwhitneyu(r, ref, alternative='two-sided').pvalue
    
    std = np.std(r)
    ss = '*' if pvalue <= 0.05 else ''
    
    if i == 0:
        pvalue = ''
    elif pvalue < .001:
        pvalue = f'$<.001$*'
    elif pvalue < .05:
        pvalue = f'${f"{pvalue:0.3f}"[1:]}$*'
    else:
        pvalue = f'${f"{pvalue:0.3f}"[1:]}$'

    if EXP_TAG == "init":
        init = c['init_denominator'] if 'init_denominator' in c else 'Reference'
        resultsForTable[i] = {'Init $\\alpha$': init, 'Top 5 acc.': f'${np.mean(r) * 100:.2f}\pm{std * 100:.1f}$', 'p-value': pvalue}
    
    elif EXP_TAG == "feats":
        feats = c['pos']['3'] if 'pos' in c else 'Reference'
        resultsForTable[i] = {'C_{f}': feats, 'Top 5 acc.': f'${np.mean(r) * 100:.2f}\pm{std * 100:.1f}$', 'p-value': pvalue}
    
    elif EXP_TAG == "metrics":
        order = 'Reference'
        if 'order' in c:
            order = convertMetric(c['order'])
        resultsForTable[i] = {'Metric': order, 'Top 5 acc.': f'${np.mean(r) * 100:.2f}\pm{std * 100:.1f}$', 'p-value': pvalue}

df = pd.DataFrame.from_dict(resultsForTable, orient='index')
if EXP_TAG == "metrics":
    df = df.sort_values(by='Top 5 acc.')
print(df.to_latex(index=False, escape=False, column_format='rrr'))

In [ ]:
LIMI = None
LIMS = None

In [ ]:
df = pd.DataFrame(np.sort(results, axis=1)[:, LIMI:LIMS, -1].T)

In [ ]:
palette=[]
for i, c in enumerate(configs):
    if c['type'] == 'reference':
        palette.append((1, 1, 1))
    else:
        p = int((1 - pvalues[i]) * 100)
        p = np.clip(p, 0, 99)
        palette.append(sns.color_palette('plasma', n_colors=100)[p])

In [ ]:
def plot_errors(ref, r, x):
    y = np.mean(r, axis=1)
    std = np.std(r, axis=1)
    
    ref_y = np.repeat(np.mean(ref, axis=0), r.shape[0], 0)
    ref_std = np.repeat(np.std(ref, axis=0), r.shape[0], 0)
    
    plt.xscale('log')
    
    plt.plot(x, ref_y, color='#4FCC1B')
    plt.fill_between(x, ref_y-ref_std, ref_y+ref_std,
        alpha=0.5, edgecolor='#4FCC1B', facecolor='#98FF48')
    
    plt.plot(x, y, color='#CC4F1B')
    plt.fill_between(x, y-std, y+std,
        alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')

    plt.show()

In [ ]:
def orness(m):
    if type(m) is not np.ndarray:
        m = m.numpy()
    mask = np.repeat(((m.shape[1] - np.arange(m.shape[1])) / m.shape[1]).reshape(1, -1), m.shape[0], axis=0)
    orness = np.sum(np.multiply(m, mask), axis=1)
    return orness

In [ ]:
def dispersion(m):
    if type(m) is not np.ndarray:
        m = m.numpy()
    m = np.clip(m, 0.000001, 1)
    dispersion = -np.sum(np.multiply(m, np.log(m)), axis=1)
    return dispersion

In [ ]:
plt.plot([10, 20 , 30])
plt.show()

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    "pgf.preamble": "\n".join([
         "\\usepackage{amsmath}"
    ]),
    'pgf.rcfonts': False,
    'figure.dpi': 300,
    'legend.fontsize': 'small',
    'axes.labelsize': 'small',
    'axes.titlesize': 'small',
    'xtick.labelsize': 'small',
    'ytick.labelsize': 'small',
    'legend.markerscale': 0.6,
    'legend.handlelength': 1.0
})
plt.tight_layout()

plt.plot([10, 20 , 30])
plt.show()

plt.close('all')

In [ ]:
if EXP_TAG == 'init':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.stack(newmats, axis=0)]

    mats = np.stack(mats, axis=0)
    
    filt_configs = configs[1:]

    meanmats = np.mean(mats, axis=1)
    
    assert len(filt_configs) == meanmats.shape[0]
    
    ornessess = np.empty((len(filt_configs), meanmats.shape[1]))
    denominators = []
    for i in range(meanmats.shape[0]):
        denominators += [filt_configs[i]['init_denominator']]
        ornessess[i, :] = orness(meanmats[i,...])
    
    fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(filt_configs)))))
    ax.plot(ornessess.T)
    ax.legend(denominators, ncol=3)
    ax.set_xlabel('Feature')
    ax.set_ylabel('Orness')
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_init_orness.pgf')
    
    dispersions = np.empty((len(filt_configs), meanmats.shape[1]))
    denominators = []
    for i in range(meanmats.shape[0]):
        denominators += [filt_configs[i]['init_denominator']]
        dispersions[i, :] = dispersion(meanmats[i,...])
    
    fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(filt_configs)))))
    ax.plot(dispersions.T)
    ax.legend(denominators, ncol=3)
    ax.set_xlabel('Feature')
    ax.set_ylabel('Dispersion')
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_init_dispersion.pgf')

In [ ]:
if EXP_TAG == 'feats':
    results_filt = np.concatenate((results[:1, ...], results[3:, ...]), axis=0)
    results_filt = results
    
    palette=[(1, 1, 1)]
    for i in range(results_filt.shape[0] - 1):
        palette.append(sns.cubehelix_palette(rot=-.5, start=.5, dark=0.5, n_colors=results_filt.shape[0] - 1)[i])

    df = pd.DataFrame(np.sort(results_filt, axis=1)[:, :, -1].T, 
                      columns=['Ref',1,2,4,8,16,32,64,128,256,512][:results_filt.shape[0]])
    df = df * 100
    fig, ax = plt.subplots()
    sns.violinplot(data=df, orient='h', palette=palette, linewidth=0.8)
    ax.set_xlabel('Top-5 Accuracy')
    plt.show()
    
    fig.set_size_inches(w=3.5, h=2.6)
    plt.tight_layout()
    plt.savefig(f'latex_figures/exp_feats_accuracies.pgf')

In [ ]:
if EXP_TAG == 'feats':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
            for file in os.listdir(REF_MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(REF_MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    nfeats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.mean(np.stack(newmats, axis=0), axis=0)]
            nfeats.append(mats[-1].shape[0])
    
    filt_configs = configs[1:]

    fig, ax = plt.subplots()
    plt.gca().set_ylim(0, 1)
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(mats)))))
    plt.xscale("log")
    ax.set_xlabel('Feature')
    ax.set_ylabel('Orness')
        
    for i in range(0, len(mats)):
        o = orness(mats[i])
        if i == 0:
            ax.plot([1, 1.001], [o, o])
        else:
            x = np.arange(o.shape[0]) + 1
            ax.plot(x, o)
    ax.legend(nfeats, ncol=3, title='$C_{f}$', prop={'size': 6})
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_feats_orness.pgf')
    
    fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(mats)))))
    plt.xscale("log")
    ax.set_xlabel('Feature')
    ax.set_ylabel('Dispersion')
        
    for i in range(0, len(mats)):
        d = dispersion(mats[i])
        if i == 0:
            ax.plot([1, 1.001], [d, d])
        else:
            x = np.arange(d.shape[0]) + 1
            ax.plot(x, d)
    ax.legend(nfeats, ncol=3, title='$C_{f}$', prop={'size': 6}, loc='lower center')
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_feats_dispersion.pgf')

In [ ]:
if EXP_TAG == 'metrics':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
            for file in os.listdir(REF_MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(REF_MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.stack(newmats, axis=0)]

    mats = np.stack(mats, axis=0)

    meanmats = mats[:, 0, ...]

    filt_configs = configs[1:]
    
    assert len(filt_configs) == meanmats.shape[0]
    
    fig, ax = plt.subplots(2, 3, sharex=True, sharey=True)
    fig.set_size_inches(w=7.1, h=5.1)

    fig.text(0.5, 0, 'Sorted input features', ha='center')
    fig.text(0, 0.5, 'Output features', va='center', rotation='vertical')
    
    for i in range(meanmats.shape[0]):
        order = convertMetric(filt_configs[i]['order'])
        ax.flat[i].set_title(order)
        ax.flat[i].imshow(meanmats[i,...], vmin=np.min(meanmats), vmax=np.max(meanmats)*0.3)
    
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_metrics_matrixes.pgf', dpi=600)

In [ ]:
if EXP_TAG == 'metrics':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
            for file in os.listdir(REF_MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(REF_MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.stack(newmats, axis=0)]

    mats = np.stack(mats, axis=0)
    
    filt_configs = configs[1:]

    meanmats = np.mean(mats, axis=1)
    
    assert len(filt_configs) == meanmats.shape[0]
    
    ornessess = np.empty((len(filt_configs), meanmats.shape[1]))
    dispersions = np.empty((len(filt_configs), meanmats.shape[1]))
    orders = []
    for i in range(meanmats.shape[0]):
        order = convertMetric(filt_configs[i]['order'])
        orders += [order]
        ornessess[i, :] = orness(meanmats[i,...])
        dispersions[i, :] = dispersion(meanmats[i,...])
        
    print(ornessess.T.shape)
    print(np.argsort(np.mean(ornessess.T, axis=0)).shape)
    plotorder = np.argsort(np.mean(ornessess.T, axis=0))[::-1]
    print(orders)
    print(plotorder)
    
    fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(1, 0, len(filt_configs)))))
    ax.plot(ornessess.T[:, plotorder])
    ax.legend(np.array(orders)[plotorder], prop={'size': 6}, ncol=2)
    ax.set_xlabel('Feature')
    ax.set_ylabel('Orness')
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_metrics_orness.pgf')
    
    print(plotorder)
    
    fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(1, 0, len(filt_configs)))))
    ax.plot(dispersions.T[:, plotorder])
    ax.legend(np.array(orders)[plotorder], prop={'size': 6}, ncol=2)
    ax.set_xlabel('Feature')
    ax.set_ylabel('Dispersion')
    
    fig.set_size_inches(w=3.5, h=2.5)
    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_metrics_dispersion.pgf')

In [ ]:
if EXP_TAG == 'metrics':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
            for file in os.listdir(REF_MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(REF_MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.stack(newmats, axis=0)]

    mats = np.stack(mats, axis=0)

    meanmats = np.mean(mats, axis=1)
        
    filt_configs = configs[1:]
    
    assert len(filt_configs) == meanmats.shape[0]
    
    show = list(range(0,64,16)) + [63]
    
    fig, ax = plt.subplots(2, 3, sharex=True, sharey=True)
    fig.set_size_inches(w=7.1, h=5.1)
    
    fig.text(0.5, 0, 'Sorted input features (i)', ha='center')
    fig.text(0, 0.5, '$w_{i}$', va='center', rotation='vertical')
    
    for i in range(meanmats.shape[0]):
        order = convertMetric(filt_configs[i]['order'])
        ax.flat[i].set_title(order)
        
        legend = list(np.array(show) + 1)
        ornesses = orness(meanmats[i, show, :])
        for j in range(len(legend)):
            legend[j] = f"{legend[j]} (orness ${ornesses[j]:.2f}$)"
        
        ax.flat[i].set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(show)))))
        ax.flat[i].plot(meanmats[i, show, :].T, linewidth=0.8)
        ax.flat[i].legend(legend,
                          title='Feature',
                          prop={'size': 6}, 
                          ncol=1,
                          loc='upper center')

    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_metrics_samples.pgf', dpi=600)

In [ ]:
def bin_owa(n, t):
    w = np.empty(n)
    for i in range(1, n+1):
        w[i-1] = scipy.special.comb(n-1,n-i)*np.power(t, n-i)*np.power(1-t, i-1)
    return w

In [ ]:
def sta_owa(n, t, alpha=0.5):
    w = np.empty(n)
    for i in range(1, n+1):
        p1 = np.prod(t + np.arange(0, n-i-1+1)*alpha)
        p2 = np.prod(1 - t + np.arange(0, i-2+1)*alpha)
        p3 = np.prod(1 + np.arange(0, n-2+1)*alpha)
        w[i-1] = scipy.special.comb(n-1,n-i)*p1*p2/p3
    return w

In [ ]:
if EXP_TAG == 'metrics':
    def get_matrixes(config):
        if 'constrainmode' in config['config']:
            constrain = WeightConstraint(config['config']['constrainmode']).apply
        else:
            constrain = WeightConstraint('full_owa').apply
        mats = []
        for it in config['results']:
            itid= it['id']
            for file in os.listdir(MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
            for file in os.listdir(REF_MATRIX_FOLDER):
                if file.startswith(itid):
                    path = os.path.join(REF_MATRIX_FOLDER, file)
                    raw_matrix = torch.Tensor(np.load(path, allow_pickle=True))
                    matrix = constrain(raw_matrix)
                    mats += [matrix[np.argsort(orness(matrix)),:]]
        return mats

    mats = []
    for f in full_configs:
        newmats = get_matrixes(f)
        if len(newmats) > 0:
            mats += [np.stack(newmats, axis=0)]

    mats = np.stack(mats, axis=0)

    meanmats = mats[0:1, 0, ...]
        
    filt_configs = [configs[1]]
    
    assert len(filt_configs) == meanmats.shape[0]
    
    show = list(range(0,64,8)) + [63]
    show = [0, 4, 11, 14, 53, 63]
    
    fig, ax = plt.subplots(2, 3, sharex=True, sharey=False)
    fig.set_size_inches(w=7.1, h=5.1)
    
    fig.text(0.5, 0, 'Sorted input features (i)', ha='center')
    fig.text(0, 0.5, '$w_{i}$', va='center', rotation='vertical')
    
    legend = ['Learned operator',
             'Binomial OWA',
             'Stancu OWA ($\\alpha =0.15$)',
             'Stancu OWA ($\\alpha =0.3$)',
             'RIM quantifier (exponential)']
    
    for i in range(6):
        raw_owa = meanmats[0, show[i], :].T
        
        o = orness(meanmats[0, show[i]:show[i]+1, :])
        
        w = 5
        raw_owa = np.convolve(raw_owa, np.ones(w)/w, 'valid')

        ax.flat[i].set_title(f"Feature {show[i]+1} (orness {o[0]:.2f})")
        
        ax.flat[i].plot(raw_owa, linewidth=1)
        ax.flat[i].plot(
            bin_owa(raw_owa.shape[0], o), 
            linewidth=0.6)
        ax.flat[i].plot(
            sta_owa(raw_owa.shape[0], o, alpha = 0.15),
            linewidth=0.6)
        ax.flat[i].plot(
            sta_owa(raw_owa.shape[0], o, alpha = 0.3),
            linewidth=0.6)
        ax.flat[i].plot(
            get_parametric_owa(raw_owa.shape[0], lambda x: rim_generator_for_orness(x, o)),
            linewidth=0.6)
        
        ax.flat[i].legend(legend,
                          prop={'size': 6}, 
                          ncol=1,
                          loc='upper center')
    


    plt.tight_layout()
    fig.show()
    plt.savefig(f'latex_figures/exp_metrics_samples.pgf', dpi=600)